In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [4]:
fake = pd.read_csv('Fake.csv')
true = pd.read_csv('True.csv')


In [5]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [6]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [7]:
# inserting a column class and value 0 for fake news and value 1 for true news

fake['class'] = 0
true['class'] = 1

In [9]:
#merging fake and true datasets

merged_df = pd.concat([fake,true], axis = 0)


In [13]:
merged_df.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [15]:
#removing the unnecessary columns from the dataframe

df = merged_df.drop(['title', 'subject', 'date'], axis = 1)

In [16]:
df.head()

,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


In [17]:
#checking for null values
df.isnull().sum()

text     0
class    0
dtype: int64

In [19]:
#shuffling the dataframe
df = df.sample(frac = 1)

In [21]:
df.head()

,text,class
13606,CARACAS (Reuters) - Venezuela systematically ...,1
485,"Remember the name Felix Sater, because it s go...",0
2074,Donald Trump just lost another one of his prop...,0
6528,(Reuters) - The following people are mentioned...,1
4930,"After being fired from the Trump campaign, Tru...",0


In [22]:
#resetting the index column and dropping the index column
df.reset_index(inplace = True)
df.drop(['index'], axis = 1, inplace = True)

In [23]:
df.columns

Index(['text', 'class'], dtype='object')

In [24]:
df.head()

,text,class
0,CARACAS (Reuters) - Venezuela systematically ...,1
1,"Remember the name Felix Sater, because it s go...",0
2,Donald Trump just lost another one of his prop...,0
3,(Reuters) - The following people are mentioned...,1
4,"After being fired from the Trump campaign, Tru...",0


In [25]:
#defining a function to process the texts and removing unwanted symbols and punctuations

def textop(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text    

In [26]:
# applying the function on all texts in the text column

df['text'] = df['text'].apply(textop)

In [27]:
df.head()

,text,class
0,caracas reuters venezuela systematically ...,1
1,remember the name felix sater because it s go...,0
2,donald trump just lost another one of his prop...,0
3,reuters the following people are mentioned...,1
4,after being fired from the trump campaign tru...,0


In [28]:
x = df['text']
y = df['class']

In [29]:
#splitting for training and testing

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.25)

In [71]:
#vectorizing the texts using tfidfvectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer()
xv_train = vec.fit_transform(x_train)
xv_test = vec.transform(x_test)

In [74]:
# using logistic regression

from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [75]:
pred_lr = LR.predict(xv_test)

In [76]:
LR.score(xv_test, y_test)

0.9856570155902005

In [77]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5899
           1       0.98      0.99      0.98      5326

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [78]:
# using Decision Tree Classifier

from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [79]:
pred_dt = DT.predict(xv_test)

In [80]:
DT.score(xv_test, y_test)

0.995456570155902

In [81]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5899
           1       1.00      0.99      1.00      5326

    accuracy                           1.00     11225
   macro avg       1.00      1.00      1.00     11225
weighted avg       1.00      1.00      1.00     11225

